### os.listdir()
用于返回指定的文件夹包含的文件或文件夹的名字的列表。
語法: os.listdir(path)

In [2]:
import os

In [5]:
dirs = os.listdir("../brain_tumor_dataset/no")
print(dirs)

['30 no.jpg', '22 no.jpg', '41 no.jpg', '14 no.jpg', 'no 10.jpg', '18 no.jpg', 'no 9.png', 'no 7.jpeg', 'no 8.jpg', 'no.jpg', '3 no.jpg', '43 no.jpg', '20 no.jpg', '32 no.jpg', '2 no.jpeg', 'N15.jpg', 'no 99.jpg', 'no 98.jpg', 'No11.jpg', '7 no.jpg', '12 no.jpg', 'No13.jpg', '44no.jpg', 'N16.jpg', 'N17.jpg', '28 no.jpg', '36 no.jpg', 'No12.jpg', '47 no.jpg', '24 no.jpg', 'No16.jpg', '10 no.jpg', '26 no.jpg', '45 no.jpg', 'no 923.jpg', '38 no.jpg', 'No17.jpg', '49 no.jpg', '34 no.jpg', '1 no.jpeg', 'no 100.jpg', 'No15.jpg', '5 no.jpg', 'N11.jpg', 'no 89.jpg', 'No14.jpg', '9 no.jpg', 'No19.jpg', 'no 90.jpg', 'N20.JPG', 'N21.jpg', 'No18.jpg', '15 no.jpg', '19 no.jpg', 'N22.JPG', 'no 92.jpg', '31 no.jpg', '40 no.jpg', '23 no.jpg', '17 no.jpg', 'N26.JPG', 'no 96.jpg', 'no 97.jpg', '21 no.jpg', '42 no.jpg', 'No22.jpg', '33 no.jpg', '50 no.jpg', 'No20.jpg', 'no 95.jpg', 'N19.JPG', 'no 94.jpg', 'No21.jpg', '37 no.jpg', 'N6.jpg', '29 no.jpg', '25 no.jpg', 'no 91.jpeg', 'no 3.jpg', '46 no.jpg', 